# 08 - Memory: RAM

## Introduction

**RAM (Random Access Memory)** is the main memory of our computer. Unlike registers (which are addressed by number), RAM is addressed by memory address, allowing us to store much more data.

Our computer will have:
- **256 bytes** of RAM
- **8-bit addresses** (0x00 to 0xFF)
- **8-bit data width**

## Learning Objectives

1. Understand how memory is organized
2. Learn about memory read/write operations
3. Build a RAM module for our computer

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root))

from computer import int_to_bits, bits_to_int
from typing import List

print("Setup complete!")

## Memory Architecture

```mermaid
flowchart TB
    Address["Address[7:0]"] --> RAM
    DataIn["Data[7:0]"] --> RAM
    WE["Write Enable"] --> RAM
    
    subgraph RAM[RAM 256 bytes]
        direction TB
        M0["0x00"] --- M1["0x01"] --- M2["0x02"] --- M3["0x03"]
        M4["..."]
        MF["0xFC"] --- MFD["0xFD"] --- MFE["0xFE"] --- MFF["0xFF"]
    end
    
    RAM --> DataOut["Data Out[7:0]"]
```

## Exercise 1: RAM Module

In [ ]:
class RAM:
    """256-byte RAM with 8-bit addressing."""

    def __init__(self, size: int = 256):
        self.size = size
        self.memory = [[0] * 8 for _ in range(size)]

    def _addr_to_index(self, address: List[int]) -> int:
        """Convert 8-bit address to integer index.

        Args:
            address: 8-bit address (LSB first)

        Returns:
            Integer index 0-255
        """
        # YOUR CODE HERE
        pass

    def read(self, address: List[int]) -> List[int]:
        """Read a byte from memory.

        Args:
            address: 8-bit address

        Returns:
            8-bit data at that address
        """
        # YOUR CODE HERE
        pass

    def write(self, address: List[int], data: List[int], enable: int) -> None:
        """Write a byte to memory.

        Args:
            address: 8-bit address
            data: 8-bit data to write
            enable: Write enable (1 to write, 0 to ignore)
        """
        # YOUR CODE HERE
        pass


# Test
ram = RAM()

# Write some values
addr1 = int_to_bits(0x10, 8)
ram.write(addr1, int_to_bits(42, 8), 1)

addr2 = int_to_bits(0x20, 8)
ram.write(addr2, int_to_bits(100, 8), 1)

# Read back
print(f"Memory[0x10] = {bits_to_int(ram.read(addr1))}")
print(f"Memory[0x20] = {bits_to_int(ram.read(addr2))}")
print(f"Memory[0x00] = {bits_to_int(ram.read(int_to_bits(0, 8)))} (unwritten)")

## Exercise 2: Loading Programs

We need a way to load programs into memory. Add a `load_program` method:

In [ ]:
class RAM:
    """256-byte RAM with 8-bit addressing."""

    def __init__(self, size: int = 256):
        self.size = size
        self.memory = [[0] * 8 for _ in range(size)]

    def _addr_to_index(self, address: List[int]) -> int:
        return sum(bit << i for i, bit in enumerate(address))

    def read(self, address: List[int]) -> List[int]:
        idx = self._addr_to_index(address)
        if 0 <= idx < self.size:
            return self.memory[idx].copy()
        return [0] * 8

    def write(self, address: List[int], data: List[int], enable: int) -> None:
        if enable == 1:
            idx = self._addr_to_index(address)
            if 0 <= idx < self.size:
                self.memory[idx] = data.copy()

    def load_program(self, program: List[List[int]], start_addr: int = 0) -> None:
        """Load a program into memory.

        Args:
            program: List of bytes (each byte is 8 bits)
            start_addr: Starting address (integer)
        """
        # YOUR CODE HERE
        pass

    def dump(self, start: int = 0, end: int = 16) -> str:
        """Dump memory contents as hex string.

        Args:
            start: Starting address
            end: Ending address (exclusive)
        """
        # YOUR CODE HERE
        pass


# Test loading a simple program
ram = RAM()

# Program: [0x12, 0x34, 0x56, 0x78]
program = [
    int_to_bits(0x12, 8),
    int_to_bits(0x34, 8),
    int_to_bits(0x56, 8),
    int_to_bits(0x78, 8),
]
ram.load_program(program, start_addr=0)

print("Memory dump:")
print(ram.dump(0, 8))

## Memory Map

In our 8-bit computer, we'll organize memory like this:

| Address Range | Use |
|--------------|-----|
| 0x00 - 0x7F | Program code |
| 0x80 - 0xEF | Data |
| 0xF0 - 0xFF | Stack (if needed) |

This is just a convention - our simple RAM treats all addresses equally.

In [ ]:
# Visualize memory with a simple demo
ram = RAM()

# Store some data values
for i in range(8):
    addr = int_to_bits(0x80 + i, 8)
    data = int_to_bits(i * 10, 8)
    ram.write(addr, data, 1)

print("Data section (0x80-0x87):")
for i in range(8):
    addr = int_to_bits(0x80 + i, 8)
    val = bits_to_int(ram.read(addr))
    print(f"  0x{0x80 + i:02X}: {val:3d} (0x{val:02X})")

## Copy Your Implementation

Once your code works, copy the `RAM` class to `src/computer/memory.py`

## Validation

In [ ]:
from utils.checker import check

check("memory")

## Summary

We've built the main memory for our computer:

| Feature | Specification |
|---------|---------------|
| Size | 256 bytes |
| Address width | 8 bits |
| Data width | 8 bits |
| Operations | read, write, load_program |

### Key Concepts

1. RAM stores data at **addressable locations**
2. **Read** is always enabled (combinational)
3. **Write** requires write enable signal
4. Programs are loaded starting at address 0

### Phase 2 Complete!

We've now built all the state/storage elements:
- Flip-flops (single bits)
- Registers (8 bits, fast access)
- Counters (auto-increment)
- RAM (256 bytes, addressable)

### What's Next?

In Phase 3, we'll design the **control logic**:
- Clock and timing
- Instruction Set Architecture
- Instruction Decoder
- Control Unit